
add an env file with:
OPENAI_API_KEY = ...
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=...

to activate env cd to the project folder and run :

source env/bin/activate

In [1]:
# pip install -r requirements.txt

In [2]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [4]:
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=512)
image = encode_image("3.png")




In [5]:

msg = chain.invoke(
    [   AIMessage(
        content="You are a useful bot that is especially good at OCR from images for math formulas and graphs"
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "solve"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)